# Introduction to Social Media: Twitter

-----

When looking for data to use for text data processing, one of the more popular data sources is [Twitter][tw]. In this notebook, we introduce the Twitter API, and demonstrate how to use the Twitter API from within a Python gram to acquire and process tweets, or Twitter messages. First, we review the mechanisms by which an application authenticates with Twitter. Next, we discuss different techniques for interacting with the twitter data stream by using the twitter API. Finally, we construct a tweet sentiment analysis pipeline before applying this pipeline to new tweets from a specific user.

-----
[tw]: https://www.twitter.com

## Table of Contents

[Python and Twitter](#Python-and-Twitter)

[Reading Twitter Data](#Reading-Twitter-Data)

[Obtaining Tweets](#Obtaining-Tweets)

[Searching for Tweets](#Searching-for-Tweets)

[Trending Topics](#Trending-Topics)

[Twitter Text Analysis](#Twitter-Text-Analysis)

- [Blind Testing](#Blind-Testing)
- [Classifying New Tweets](#Classifying-New-Tweets)
-----

Before proceeding with the rest of this notebook, we first include the notebook setup code.

-----

In [5]:
# Standard imports
import numpy as np
import nltk
import pprint

-----

[[Back to TOC]](#Table-of-Contents)

## Python and Twitter

To work with the Twitter API from within a Python program, we need a Python library that wraps the official [Twitter API][twapi]. There are a number of different Python libraries that provide this capability, we will use the [tweepy][tpy] library, which is fairly popular and provides a fairly complete interface.

The full Twitter API is large and robust (and continuous to evolve), for this course we will restrict our attention to several basic concepts, namely authenticating to Twitter, searching for Tweets, and digesting the messages.

----
[twapi]: https://dev.twitter.com
[tpy]: http://www.tweepy.org

In [6]:
# Import python twitter library
import tweepy as tw

-----

[[Back to TOC]](#Table-of-Contents)

## Reading Twitter Data

To read Twitter data, you need to first need to be a registered Twitter user and you need to create a new _Twitter Application_ in order to obtain credentials for connecting to Twitter and querying to the Twitter data. You create (and later manage) Twitter applications by visiting the [Twitter Application Management](https://apps.twitter.com) website.

![Twitter App Sign-in](images/twitter-app-signin.png)

At this point you need to authenticate with Twitter, if you are already logged in to Twitter on your computer (for instance by using the Twitter website) you should already be authenticated. If you are not authenticated, click the _sign in_ link to be directed to the Twitter sign in page where you can enter your credentials (if you do not have Twitter credentials, you will need to obtain a Twitter account to proceed).

![Twitter Sign-in](images/twitter-signin.png)

After you have been authenticated, you will be redirected to the Twitter apps page. If you have never created a Twitter application, you will have nothing listed. To create a new application, press the _Create New App_ button, as shown in the following screenshot.

![Twitter Create App](images/twitter-create.png)

This will open up the Twitter _Create an application_ webpage, where you need to supply some basic information for your Twitter application such as an application name, description, and website.

![Twitter Application details](images/twitter-appdetails.png)

Scroll to the bottom of this webpage where the **Developer Agreement** is located. Following this agreement, is a check box that you should click to signify you agree to be bound by the agreement (of course you should read this to be sure you do _agree_ with it first). Following this, press the _Create your Twitter application_ button as shown in the following screenshot.

![Twitter Agree](images/twitter-agree.png)

This will create your new application, and provide you with your application webpage, which will be similar to the following screenshot. 

![Twitter App-page](images/twitter-apppage.png)

While you can control a number of application features from this webpage, the most important tasks to complete include:

1. Change your application to _read-only_ in case it is set to read-write.

2. Obtain the application **Consumer Key** and **Consumer Secret**.

3. Obtain your personal **Access Token** and **Access Token Secret**.

You should change your application read-only to ensure you don't accidentally send data out to Twitter. You change this by selecting the _Permissions_ tab and selecting _Read only_, shown in the following screenshot. To save this setting, scroll down this webpage and click the _Update Settings_ button at the bottom of the page.

![Twitter Read Only Setting](images/twitter-ro.png)

These credentials can be found by selecting the _Keys and Access Tokens_ tab, and scrolling down appropriately as shown in the following two screenshots.

![Twitter Consumer Application Credentials](images/twitter-consume.png)

![Twitter User Credentials](images/twitter-access.png)

<font color='red'> Warning: Never share these credentials with others or they will be able to fully impersonate you on Twitter! </font>

You can directly copy these credentials into your notebook, or, alternatively, save them into a file (for example by opening a terminal window and using `vim` to create a text file. In the rest of this notebook, I demonstrate this functionality by using my credentials, which I have saved into a file called `twitter.cred`. In this empty file, which is in your github repository, I have saved the following four credentials in order:

1. Access Token
2. Access Token Secret
3. Consumer Key
4. Consumer Secret

If you are using git for source code management and control, you can inform `git` to ignore changes in the `twitter.cred` file by using the following command in the current module directory:

```bash
git update-index --assume-unchanged notebooks/twitter.cred 
```

The following Code cell demonstrates how these credentials are read from the file and used to properly authenticate our application with Twitter.

-----

In [4]:
tokens = []

# Order: Access Token, Access Token Secret, Consumer Key, Consumer SecretAccess

with open("twitter.cred", 'r') as fin:
    for line in fin:
        if line[0] != '#': # Not a comment line
            tokens.append(line.rstrip('\n'))

auth = tw.OAuthHandler(tokens[2], tokens[3])
auth.set_access_token(tokens[0], tokens[1])

api = tw.API(auth)

user = api.me()

print("Twitter Screen Name: ", user.screen_name)
print("Twitter Follower Count: ", user.followers_count)

IndexError: list index out of range

-----

If the previous Code cell runs without an error, you have successfully connected to Twitter. If you are new to Twitter and are not following anyone, you can instead display the user information for a different Twitter user. For example, the following code would display my Twitter information.

```python
user = api.get_user('ProfBrunner')
```

Replacing `ProfBrunner` with any valid Twitter user id will display their information. You can find examples by looking at those Twitter users you (or `ProfBrunner`) follow. Or, alternatively, you could chose a specific twitter account; for example, to analyze the _NY Times_ Twitter account you would use the following statement:

```python
user = api.get_user('NYTimes')
```

This is demonstrated in the following Code cell.

-----

In [ ]:
user = api.get_user('nytimes')

print("Twitter Screen Name: ", user.screen_name)
print("Twitter Follower Count: ", user.followers_count)

print("\nThis user follows:\n--------------")
for friend in user.friends():
    print(friend.screen_name)

-----

At any point, you can return to your Twitter application management webpage to view your new application. You can now view and manage your existing application, or create a new application as shown in the following screenshot.

![Twitter new app management](images/twitter-manage.png)

-----

-----

<font color='red' size = '5'> Student Exercise </font>


To run the Twitter application in the preceding cells, you will need to register your own Twitter Application. To do so, complete the following steps.

1. Create a New Twitter application.

2. Save your Twitter credentials and Application credentials into the provided `twitter.cred` file.

3. Run the _tweepy_ sample code to connect to Twitter and display your Twitter user information.

Finally, try running the preceding code, but for someone famous (if you do not know the Twitter handle for someone famous, Google will be your friend). 

-----

[[Back to TOC]](#Table-of-Contents)

## Obtaining Tweets

Once you have authenticated with Twitter, you can begin to [search the Twitter stream][stw] for tweets of interest. The easiest method to get started is to being with your own (or another specific Twitter user's) own Twitter feed. To access your own Twitter feed, you can simply use your `home_timeline` to retrieve your own Tweets or Tweets from those whom you follow. This is demonstrated in the following Code cell, where we display the `text` values from the ten most recent Tweets from our timeline.

-----
[stw]: https://dev.twitter.com/rest/public/search

In [ ]:
for tweet in tw.Cursor(api.home_timeline).items(10):
    # Process a single status
    print(tweet.text) 

-----

[[Back to TOC]](#Table-of-Contents)

## Searching for Tweets

Twitter also provides the capability to search for specific tweets by using the Tweepy [`search` method][twse]. In this method, you supply a query string (and optional arguments) and are returned a list of Tweets. The query string should follow the [Twitter Search API][tsa], but basically you can search for specific text in a string by using the text of interest, you can search for a person by using the `@` character followed by their Twitter username, and hashtags by using the `#` character followed by the tag text.

-----
[twse]: http://docs.tweepy.org/en/stable/api.html#API.search
[tsa]: https://dev.twitter.com/rest/public/search

In [ ]:
# Hash Tage search: term = '#python'
# User search: term = '@nytimes'
# Keyword search: term = 'data science'
# Keyword and Sentiment: term ='data science :)' # Positive attitute

term ='data science :)'
num_tweets = 5

for tweet in tw.Cursor(api.search, q=term).items(num_tweets):
    # Process a single status
    print("Tweet ID:", tweet.id)
    print('Tweeted by ', tweet.user.screen_name)
    print("Created at ",tweet.created_at)
    print("Location: ",tweet.source)
    print('Tweet Text: ', tweet.text)
    print('-------------------------')

-----

We can view the available attributes to display by using Python `dir` method to perform introspection. In the following Code cell we explicitly remove _class_ methods to minimize the display list and focus on the items of interest. After this, we display the Tweet in its raw JSON format by accessing the `_json` attribute.

-----

In [ ]:
# Print out single tweet

pp = pprint.PrettyPrinter(indent=2, depth=2, width=80, compact=True)

tweets = api.search(q='Deloitte', rpp=1)

pp.pprint([att for att in dir(tweets) if '__' not in att])

In [ ]:
# Pick a single tweet to analyze
tweet = tweets[1]
pp.pprint([att for att in dir(tweet) if '__' not in att])

In [ ]:
# We can display the message data in JSON format
pp.pprint(tweet._json)

-----

[[Back to TOC]](#Table-of-Contents)

## Trending Topics

Twitter tracks tweet data to identify topics or people that are being frequently mentioned, which is known as [_trending_][twtr]. The Twitter API enables an application to obtain a list of currently trending topics. These topics include metadata that can be used to learn more about trending topics. One component of the metadata is the physical location of the trending topic. This location is encoded as a [**WOEID**][woeid], which is a Yahoo developed standard that is short for _where on the earth ID_. In the first Code cell below, we demonstrate obtaining the locations of currently trending topics before displaying these physical locations. In the second Code cell, we display the complete metadata for one location, which can be used to obtain a list of trending topics for a particular location on Earth, via the WOEID. Note that since trending topics change, this notebook will provide different results when run at different times.

----
[twtr]: https://dev.twitter.com/rest/reference/get/trends/available
[woeid]: https://developer.yahoo.com/geo/geoplanet/guide/concepts.html

In [ ]:
# Returns a JSON object that contains (a large number of) locations 
# that are currently trending.

top_display = 20
trending = api.trends_available()

# We skip first value, which is entry for the World in JSON.
for trend in trending[1:top_display]:
    print('WOEID Code ({2:d}): {0}, {1}'.format(trend['name'], \
                                                trend['country'], trend['woeid']))

In [ ]:
pp.pprint(trending[10])

In [ ]:
# We can use a WOEID to find location specific trends.
# Here we use the WOEID for the UK (from previous example)

top_display = 10

print("UK Trends")
print(10*'-')

for trends in api.trends_place(id = 23424975):
    for trend in trends["trends"][:top_display]:
        print("  {0:s}".format(trend["name"]))

-----

<font color='red' size = '5'> Student Exercise </font>


In the preceding cells, we used the twitter API to obtain tweets and to identify trending topics. Now that you have run the notebook, try making the following changes.

1. Pick a particular twitter user and search their twitter stream.
2. Pick a different location from the trending topics location list, and identify trending topics from a different WOEID.

-----

[[Back to TOC]](#Table-of-Contents)


## Twitter Text Analysis

We can now develop a text analysis project that uses twitter data. To simplify the application, we will use the NLTK twitter corpus. Otherwise, we would need a separate notebook to obtain the necessary tweets (because of the twitter rate limitation). The [NLTK twitter corpus][ntw] includes thirty thousand tweets retrieved from the twitter streaming API, the data have been cached on our course JupyterHub server. The tweets were explicitly selected from a recent election in the United Kingdom and one-third of them have been classified into positive or negative (with equal numbers of each). With these tweets, we can build a classification pipeline to perform sentiment analysis on twitter data.

In the following Code cells, we first obtain the tweets, build the NumPy arrays for our classification pipeline, before constructing and testing this simple sentiment analysis text analysis application.

-----
[ntw]: http://www.nltk.org/howto/twitter.html#Using-a-Tweet-Corpus

In [ ]:
tws = nltk.corpus.twitter_samples

pos_tweets = np.array(tws.strings('positive_tweets.json'))
neg_tweets = np.array(tws.strings('negative_tweets.json'))

pos_labels = np.ones(pos_tweets.shape[0])
neg_labels = np.zeros(neg_tweets.shape[0])

targets = np.concatenate((pos_labels, neg_labels), axis=0)
data = np.concatenate((pos_tweets, neg_tweets), axis = 0)

print('{0} Positive Tweets'.format(pos_tweets.shape[0]))
print('{0} Negative Tweets'.format(neg_tweets.shape[0]))

-----

We will employ 75% of the data for training, with 25% held out for validation. The classification pipeline will use a simply tokenizer to build a document-term matrix before applying a Naive Bayes classifier. The tokenizer will use _English_ stop words, will convert the text to all lowercase, and includes both unigrams and bigrams. Overall, this simple classification pipeline gives reasonable results.

-----

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    data, targets, test_size=0.25, random_state=23)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics

tools = [('cv', CountVectorizer()), ('nb', MultinomialNB())]
pclf = Pipeline(tools)


# Lowercase, English Stop Words, and unigrams and bigrams.
pclf.set_params(cv__stop_words = 'english', \
                cv__ngram_range=(1,2), \
                cv__lowercase=True)

pclf.fit(x_train, y_train)
y_pred = pclf.predict(x_test)
print(metrics.classification_report(y_test, y_pred, target_names = ['Positive', 'Negative']))

-----

[[Back to TOC]](#Table-of-Contents)

### Blind Testing

We can use the remaining twenty thousand tweets in the NLTK corpus for blind testing. We first obtain the tweets as a NumPy array, before applying our sentiment analysis pipeline. Finally, we display the relative numbers of positive and negative classifications and we display examples of both positive and negative classified tweets.

-----

In [ ]:
unknown_tweets = np.array(tws.strings('tweets.20150430-223406.json'))
unknown_pred = pclf.predict(unknown_tweets)

unknown_pos = unknown_tweets[unknown_pred == 1]
unknown_neg = unknown_tweets[unknown_pred == 0]

In [ ]:
tweet_idx = 101

print(f'{unknown_tweets.shape[0]} tweets to classify.')
print(f'{unknown_pos.shape[0]} tweets classified as positive.')
print(f'{unknown_neg.shape[0]} tweets classified as negative.')

print(75*'-')
print('Sample Positve Tweet:')
print(75*'-')
print(unknown_pos[tweet_idx])

print(75*'-')
print('Sample Negatve Tweet:')
print(75*'-')
print(unknown_neg[tweet_idx])

-----

[[Back to TOC]](#Table-of-Contents)

### Classifying New Tweets

We can now combine everything covered in this notebook in order to apply our trained sentiment analysis pipeline on new twitter data. For this, we pick _random_ user, in this case CNN Political Twitter Feed (note this wasn't completely random. The training data was obtained from a similar feeds). We first obtain tweets from this user before creating the NumPy arrays to use with our scikit learn classifier. Finally, we display the results of this sentiment analysis classifier. Note, since the tweets will change over time, the results presented in this notebook will also change.

-----

In [ ]:
newtweets = api.user_timeline(screen_name='@CNNPolitics', include_rts=False, count=100)
                           
print(f'We obtained a sample of {len(newtweets)} tweets')

In [ ]:
messages = []

for tweet in newtweets:
    messages.append(tweet.text)
    
new_tweets = np.array(messages)
new_pred = pclf.predict(new_tweets)

new_pos = new_tweets[new_pred == 1]
new_neg = new_tweets[new_pred == 0]

In [ ]:
# Pick a tweet index
tweet_idx = 13

print(f'{new_tweets.shape[0]} tweets to classify.')
print(f'{new_pos.shape[0]} tweets classified as positive.')
print(f'{new_neg.shape[0]} tweets classified as negative.')

print(75*'-')
print('Sample Positve Tweet:')
print(75*'-')
print(new_pos[tweet_idx])

print(75*'-')
print('Sample Negatve Tweet:')
print(75*'-')
print(new_neg[tweet_idx])

-----

<font color='red' size = '5'> Student Exercise </font>


In the preceding cells, we build a sentiment analysis classification pipeline by using the NLTK corpus before applying it to new tweets. Now that you have run the notebook, try making the following changes.

1. Modify the pipeline parameters, for example, apply stemming, change the `max_features`, or the number of n-grams. Can you improve the classification results on the validation data?

2. Change the type of classification algorithm (e.g., random forest or SVC with regularization). Can you improve the classification results on the validation data?

3. Using your new classification pipeline, examine the performance on the current twitter user. Look at other tweets, does the performance improve?

4. Try classifying tweets from a different user, either an _election_ type feed or a popular figure. By looking at select tweets and their classification, comment on how your classifier performs?

Finally, why do you think the classification pipeline performs in the manner that is does (i.e., why are some tweets classified negative/positive)? Feel free to use the class forums.

-----

## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

1. Wikipedia article on [twitter][wt]  
1. Wikipedia article on [Social Media][wsm]  
1. Twitter, [official documentation][tod]  
1. Map of a [twitter status object][mtso]  
1. [Tweepy][twd]: Python Twitter client documentation (Getting Started and Authentication Tutorial)  
1. [Using nltk][unt] with twitter (note uses a different twitter client library)  
1. Blog demonstrating [twitter access via URLs][tu]  
1. Blog article on collecting tweets by using [streaming api][tsa]  
1. **Chapter 1: Mining Twitter** from _Mining the Social Web_, [Jupyter notebook][msw1]  
1. **Chapter 9: Twitter Cookbook** from _Mining the Social Web_, [Jupyter notebook][msw1]  

-----

[wt]: https://en.wikipedia.org/wiki/Twitter
[wsm]: https://en.wikipedia.org/wiki/Social_media

[twd]: http://tweepy.readthedocs.org
[tod]: https://dev.twitter.com/overview/documentation

[unt]: http://www.nltk.org/howto/twitter.html

[tsa]: http://badhessian.org/2012/10/collecting-real-time-twitter-data-with-the-streaming-api/
[tu]: http://nealcaren.web.unc.edu/pizza-twitter-and-apis/

[msw1]: https://github.com/ptwobrussell/Mining-the-Social-Web-2nd-Edition/blob/master/ipynb/Chapter%201%20-%20Mining%20Twitter.ipynb
[msw9]: https://github.com/ptwobrussell/Mining-the-Social-Web-2nd-Edition/blob/master/ipynb/Chapter%209%20-%20Twitter%20Cookbook.ipynb
[mtso]: http://online.wsj.com/public/resources/documents/TweetMetadata.pdf

**&copy; 2017: Robert J. Brunner at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode